# 大模型微调

一、大模型微调简介*

二、微调示例**

三、微调模型的数据准备**

四、微调在大模型训练中的位置***

五、PEFT方法对比 **

六、可能有害的全量微调 **

## 一、大模型微调简介

本章节内容：
- 微调解决的问题
- 微调到底是什么

### 微调解决的问题

#### 基础模型输出模式较少

**基座模型输出：**

<img src="image/预训练输出例子.png"  width="900" height="600" alt="rag-vs-finetuning-chn"/>

**针对对话能力微调后：**

<img src="image/SFT输出例子.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

#### 通用大模型缺乏领域知识/回答存在幻觉问题

问题：

<img src="image/周鲁1.png"  width="600" alt="区别"/><br>

回答：

<img src="image/周鲁2.png"  width="600" alt="区别"/><br>

### 微调到底是什么

微调（Fine-tuning）是一种机器学习技术，特别是在深度学习领域中，它指的是在一个预训练的模型基础上继续训练，以使模型适应特定的任务。预训练模型通常在大型数据集上进行训练，以学习丰富的特征表示。然后，这些模型可以用较小的、特定任务相关的数据集进行微调，以细化它们对特定问题的处理能力。

### Prompt Tuning

- Prompt Tuning 起源于GPT3 (论文：[Language Models are Few-Shot Learners](https://arxiv.org/pdf/2005.14165.pdf))，其认为超大规模的LLM只要配合好合适的模板就可以极大化地发挥其推理和理解能力。

<img src="image/prompt t.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

#### 什么是Prompt-Tuning？

我们可以把Prompt-Tuning想象成一种训练智能机器人的简化方法。通常，这个智能机器人（我们称之为预训练语言模型，比如GPT-3或BERT）已经通过阅读大量文本学到了很多知识。但是，当我们想要让它完成一个特定的任务时，比如情感分析或者文本分类，传统的方法是Fine-Tuning。这就像让机器人回学校进修，学习大量特定任务的示例。但这个过程既耗时又耗资源。

Prompt-Tuning，顾名思义，是通过设计一种“提示”（Prompt），来引导机器人利用它已经掌握的知识去完成新任务。这种方法更加高效，因为它不需要机器人学习大量新的信息。


## 二、微调示例

本章节内容：
- 微调步骤
- 微调全过程展示

### 微调的步骤通常包括以下几个方面：

- 准备特定任务的数据集：虽然预训练模型拥有广泛的知识，但它们需要通过特定任务的数据来了解特定领域的细节。
- 选择预训练模型：这通常是一个在广泛数据上已经训练过的大型模型，如在文本处理任务上的BERT或GPT，或者在图像识别任务上的ResNet。
- 模型调整(可选)：可能需要对模型架构进行一些调整，如添加或修改输出层，以适应新任务的输出需求。
- 开始微调：使用特定任务的数据集继续训练模型。这个过程可能会涉及调整**学习率**，选择合适的**损失函数**和**优化器**，以及决定哪些层的权重需要被冻结或更新。
- 评估和调优：在微调后，模型将在验证集上进行评估，以确保其性能符合预期。可能需要进行多次迭代，以找到最佳的参数和设置。

### 什么是损失函数？
我们可以把深度学习模型想象成一个试图解决问题的机器。假设，我们要教这台机器如何识别照片中的猫和狗。为了让机器能学会这个技能，我们需要给它看很多已经标记好是猫还是狗的照片，这个过程就叫做“训练”。

在训练过程中，深度学习模型会不断地去调整权重去尝试正确预测每张照片是猫还是狗。损失函数，就是用来衡量模型的预测结果和真实结果之间的差距的一种方法。你可以把它想象成一种评分规则，这个规则会告诉我们模型的预测有多准确，或者说错误有多大。

例如，如果模型预测一张猫的照片是狗，损失函数就会给出一个高分（表示错误较大）。相反，如果预测正确，损失函数的分数就会很低（表示准确度高）。我们的目标是通过调整模型参数，使得这个损失函数的分数尽可能地低，这样模型的预测准确度就会越高。

常见的损失函数有均方误差（Mean Squared Error，MSE）、交叉熵（Cross Entropy）等。不同的问题和模型可能需要使用不同的损失函数。

例如，对于二分类问题，我们通常会使用二元交叉熵损失函数。这个损失函数的计算公式如下：

$$
L(y, \hat{y}) = -\frac{1}{N} \sum_{i=1}^{N} y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)
$$

假设我们将图像分类为猫（类别 0）或狗（类别 1），那么 $y_i$ 是真实标签，$\hat{y}_i$ 是模型的预测概率。例如在某一次训练中，模型输出了一个值表示去预测一张猫的照片是狗的概率为 0.8，那么损失函数的计算公式如下：

$$
L(y, \hat{y}) = - （0 \times \log(0.8) + 1 \times \log(0.2)） = -\log(0.2) \approx 0.69
$$

而如果模型预测一张狗的照片是狗的概率为 0.8，那么损失函数的计算公式如下：

$$
L(y, \hat{y}) = - （1 \times \log(0.8) + 0 \times \log(0.2)） = -\log(0.8) \approx 0.22
$$

可以看到，当模型的预测结果和真实结果越接近，损失函数的值就越小。模型进行训练的过程就是通过不断地调整参数，使得损失函数的值总体上尽可能地小。

总之，损失函数是深度学习中一个非常关键的概念，它帮助我们了解模型在学习过程中的表现如何，并指导我们如何改进模型，使其预测结果更加准确。



### 什么是学习率？
现在让我们来聊聊深度学习中的一个非常重要的概念——学习率。

首先，想象一下你正在教一个机器人如何走路。你不可能一次就教会它所有的步骤，而是需要一步一步地调整它的动作，直到它能够流畅地行走。在这个过程中，你每次调整的幅度就好比是学习率。

刚刚你学习了损失函数，它可以告诉我们机器学习模型的预测结果和实际结果之间有多大的差距。我们的目标是减小这个差距，使模型的预测更加准确。

学习率，就是在这个优化过程中，我们调整模型参数的幅度。如果学习率太高，模型参数的调整幅度会过大，可能会导致模型的参数在参数空间中来回跳动，无法收敛到最优解。这就好比你教机器人走路时，每次调整它的步伐都太大，结果它不断摔倒。

相反，如果学习率太低，每次参数的调整幅度又太小，模型需要很长时间才能达到最优状态，甚至可能陷入局部最优解，无法取得更好的效果。这类似于你教机器人走路时，每次调整它的步伐都太小，以至于它学习的速度非常缓慢。

<img src="image/learning.png"  width="900" alt="学习率"/><br>

因此，选择一个合适的学习率非常关键，它决定了学习的速度和效率。在实际应用中，我们可能会开始时使用较大的学习率，以快速接近最佳解，然后逐渐减小学习率，以确保模型能够稳定地达到最优解。

我们总结一下，学习率就是我们在训练深度学习模型时，调整模型权重的幅度。通过合理地设置学习率，我们可以加速模型的收敛速度，提高模型的训练效率。


### 什么是优化器？
刚刚我们学习了损失函数和学习率，它们分别用来衡量模型的预测结果和实际结果之间的差距，以及调整模型参数的幅度。那么，优化器是用来做什么的呢？

在深度学习中，我们通过不断地调整模型的参数，使得损失函数的值尽可能地小，以提高模型的预测准确度。而优化器帮助我们自动地实现这个目标，它通过计算损失函数的梯度，通过某些优化算法来调整模型的参数，使得损失函数的值不断减小。

例如，我们可以使用梯度下降算法来优化模型。这个算法的基本思想是，通过计算损失函数对模型参数的梯度，然后沿着梯度的反方向调整参数，理论上这使得损失函数的值不断减小，从而使得模型的预测结果就会越来越准确。

又如，我们可以使用 Adam 优化器来优化模型。Adam 优化器是一种自适应学习率的优化算法，它可以根据梯度的大小自动调整学习率，简单来说，Adam 结合了动量（momentum）和 RMSprop 优化器的优点，通过计算梯度的一阶动量和二阶动量来动态调整每个参数的学习率，使得优化过程更高效、更稳健。

Adam 是基于梯度下降算法的，但它通过两个关键的技术来改进传统的梯度下降方法：

- 动量方法：Adam 通过计算梯度的一阶动量（即梯度的指数加权移动平均）来加速收敛。动量方法有助于在梯度方向一致时积累更快的更新，同时在梯度方向不一致时减少振荡。

- 自适应学习率：Adam 还计算梯度的二阶动量（即梯度平方的指数加权移动平均），这有助于自动调整每个参数的学习率。梯度较大的参数会有较小的更新步伐，而梯度较小的参数则会有较大的更新步伐。这种方法类似于 RMSprop 优化器，但 Adam 结合了一阶动量和二阶动量，使得优化过程更为灵活。

总而言之，优化器通过帮助我们自动计算损失函数的梯度，调整模型的参数。选择合适的优化器就可以以更高的效率和更快的速度训练模型，提高模型的性能。

<img src="image/optimizer.png"  width="600" alt="优化器"/><br>

### 准备特定任务的数据集

#### 数据集的重要指标
1. **提供高质量数据**
   
   数据质量在微调中是非常重要的。给出垃圾数据，大模型更容易生成垃圾内容；给出高质量数据，大模型就有更大概率生成高质量内容
   
   **注意：** 数据质量不是最终模型性能的唯一决定因素
2. **保证数据多样性**
   
   如果给出的数据集中有很多类似的输入输出，那么模型会倾向于记忆这种模式。当用户输入不一样的内容时，模型可能只能输出重复的内容。

3. **数据真实性（对比生成的数据）**
   虽然现在有很多工具都可以生成训练用数据，但是真实的数据在某些任务上是至关重要的，比如写作，生成的数据会有很多固定的生成模型，过多的生成数据无法保证模型可以学习到新的模式。

4. **更多的数据**
   这点没有其他三点重要（尤其是数据质量），但更多的数据在某种程度上仍然表现得比较少的数据数量更好。

#### 数据收集与重构
开源网站：https://huggingface.co/

Hugging Face提供了很多的开源数据集，但当我们想要基于自己的需求进行模型微调时仍需要自己构建数据集，这里我们不对数据来源进行限制，直接来看一下数据集应该以什么样的数据呈现。

下列数据来自[Standford Alpaca](https://github.com/tatsu-lab/stanford_alpaca)，用于为大模型提供基本的对话能力

<img src="image/alpaca.png"  width="600"  alt="alpaca"/>

<img src="image/parse_analysis.png"  width="600"  alt="alpaca-data"/>

Llama2

<img src="image/llamadata.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

GPT3

<img src="image/gpt.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

总体而言，他们都分为以下几个部分：

**instruct/prompt:**
给出一个问题或指令，例如，“世界上最高的山是什么名字？”

**input:**
给出上下文，例如回答问题的必要信息或者指定的回答问题的格式，有时候这部分会被合并到instruct/prompt

**output/completion:**
这是答案部分，针对上面的问题，答案可能是，“世界上最高的山是珠穆朗玛峰。”


#### 下面我们来看看具体的数据处理过程

In [2]:
import pandas as pd
import json

def convert_csv_to_json(csv_file_path):
    # 根据文件结构调整CSV读取参数
    try:
        # 使用pandas读取CSV文件
        # 看起来分隔符是逗号，不是制表符
        df = pd.read_csv(csv_file_path, sep=',', encoding='utf-8')
    
        # 将DataFrame转换为字典列表
        data = df.to_dict(orient='records')
    
        # 将数据格式化为所需的JSON结构
        json_data = [{"instruction": item["问题"], "output": item["回答"]} for item in data]
    except Exception as e:
        json_data = f"发生错误：{str(e)}"
    return json_data


In [37]:
df = pd.read_csv("./example.csv", sep=',', encoding='utf-8')
df

,问题,回答
0,使用定性和定量分析相结合的方法，研究并编写一份关于地震的地质、地震学和社会影响的全面报告。你...,标题：全面研究报告：地震的地质、地震学影响与社会影响\n\n一、地震的定义和理解\n\n地震...
1,我该如何在C++中评估废除夏令时对经济的影响？你能否提供一个考虑能源消耗、生产力和交通成本等...,在现实中，评估废除夏令时对经济的影响这样的复杂问题并不适合用编程语言（包括C++）来解决。这...
2,假设你正在调试一段Java代码，但突然意识到自己既是侦探又是犯罪电影中的凶手。为了解决这个谜...,"在Java代码中，有许多可能的数据类型可能引起错误。以下是几个可能的\""罪犯\""：\n\n1..."


#### pd.read_csv的内部实现原理
实际上，pandas的read_csv函数实现并不简单，它可以接受超过50个参数，这些参数控制数据的读取、解析、转换等过程。下面我们来大致看看read_csv函数的内部实现原理。

**1.函数入口**

read_csv函数在pandas/io/parsers.py中定义。调用read_csv时，会首先进行一些参数检查和预处理，然后调用内部的TextFileReader类来处理文件读取逻辑。

**2.参数解析**

read_csv函数的参数非常多，这里只列出一部分常用的参数：

- filepath_or_buffer：文件路径或文件对象
- sep：分隔符，默认为逗号
- header：指定哪一行作为列名
- names: 用于结果的列名列表。
- index_col: 指定列为索引的列编号或列名。
- usecols: 返回一个数据子集，该子集由列组成。
- dtype: 指定列的数据类型。

**3.创建TextFileReader对象**

实际上，TextFileReader是一个迭代器，它会根据chunksize参数将文件分块读取，然后对每个块进行解析。

```python
parser = TextFileReader(filepath_or_buffer, **kwds)
```

**4. 建立解析引擎**

TextFileReader内部根据文件类型和参数，选择合适的解析器。主要的解析器是PythonParser和CParserWrapper，分别对应Python实现和C实现。

```python
self._engine = self._make_engine(self.engine)

def _make_engine(
        self,
        f: FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str] | list | IO,
        engine: CSVEngine = "c",
    ) -> ParserBase:
    mapping: dict[str, type[ParserBase]] = {
            "c": CParserWrapper,
            "python": PythonParser,
            "pyarrow": ArrowParserWrapper,
            "python-fwf": FixedWidthFieldParser,
        }
    ...

    try:
        return mapping[engine](f, **self.options)
```

**5. 解析数据**

无论使用哪种解析器，都会调用相应引擎的read方法来读取文件内容并将其转换为DataFrame。以下是PythonParser的read方法的逻辑：

1. 如果存在索引名称，并且第一行的数据全为空，那么这一行就被认为是索引名称行，将被单独处理，并从数据中移除。

2. 将行数据转换为列数据。

3. 处理隐式索引。隐式索引是指没有明确标记为索引的列，但是在解析数据时被当作索引处理。

4. 将数据转换为适当的数据类型。

5. 处理日期转换。如果某一列的数据可以被解析为日期，那么这一列的数据将被转换为日期类型。

6. 创建索引。

7. 最后，返回一个元组，包含索引、列名和数据。

显然，这个元组可以最终被Pandas的DataFrame接受，从而创建一个DataFrame对象。

In [3]:
csv_file_path = "./example.csv"
json_output = convert_csv_to_json(csv_file_path)
print(json_output[0])

{'instruction': '使用定性和定量分析相结合的方法，研究并编写一份关于地震的地质、地震学和社会影响的全面报告。你的报告必须包括至少10个关于地震的独特和有趣的事实，这些事实来自各种来源，包括但不限于学术期刊、政府报告和可靠的新闻媒体。此外，你的报告应探讨地震的历史和原因，包括但不限于构造板块、断层线和地震波的作用。此外，你的报告还应分析地震对社会和经济的影响，包括但不限于对基础设施、公共卫生和应急响应系统的影响。最后，你的报告应以一系列建议为结论，为政策制定者和利益相关者提供如何减轻与地震相关的风险的建议，考虑到最新的科学研究和来自世界各地的最佳实践。你的报告应使用清晰简洁的语言编写，适当引用和参考文献。', 'output': '标题：全面研究报告：地震的地质、地震学影响与社会影响\\n\\n一、地震的定义和理解\\n\\n地震是地壳内部突然释放能量的过程，这种过程引发振动和散射地震波，从而引发的横跨数千公里的地表振动。地壳内的能量主要是由于地球板块的相互作用，尤其是构造板块以及到地震中心的断层线（定量事实1）。\\n\\n二、地震发生的原因\\n\\n1. 构造板块：地震常常在板块的边界发生，这是因为这些地区压力累积，最终导致地壳破裂。例如，太平洋板块和北美板块在西部美国的加利福尼亚州交界处，这就是著名的圣安地列斯断层线，经常发生地震（定量事实2）。\\n\\n2. 断层线：断层是地壳中的裂缝，两边的岩石发生相对位移。这种移动可导致地震，或由地震引起。\\n\\n3. 地震波：地震会产生三种地震波，即P波（初到波），S波（剪切波）和表面波。P波是最先到达地震仪的，速度最快。然后是S波，最后是表面波，它们在地震的过程中产生最大的破坏（定量事实3）。\\n\\n三、地震的影响\\n\\n1. 基础设施：地震会导致建筑物、桥梁、道路、电网、水管等基础设施破裂或倒塌，不仅导致财产损失，还可能导致人员伤亡。例如，1994年的北岭地震导致57人死亡，超过8700人受伤，直接的经济损失约为200亿美元（定量事实4）。\\n\\n2. 公共卫生：地震后的灾区常常出现供水、排水、卫生设施、医疗服务等基础设施的破坏，这可能会引发公共卫生危机，如传染病的爆发等（定量事实5）。\\n   \\n3. 应急响应系统：地震会对应急响应系统提出挑战。一方面，基础设施的损坏会影响

### 选择预训练模型

<img src="image/hfos.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

<img src="image/describe.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

### 模型调整(可选)

微调方法可以被分为全量微调和PEFT
#### 全量微调

全量微调是一类较早诞生的微调方法，全参数微调需要消耗大量的算力，实际使用起来并不方便，因此不久之后又诞生了只围绕部分参数进行微调的高效微调方法

- 全量微调的问题：

1. 高计算成本

    全面微调涉及更新大型模型的所有参数。大规模的预训练语言模型拥有数百亿甚至数千亿的参数，因此训练需要大量的计算资源。即使微调数据集相对较小，标记的数量也可能很大，计算成本会非常高昂。

2. 时间和专业知识密集型

    当模型非常大时，通常需要将计算分布到多个GPU和节点上。这需要适当的专业知识。根据模型的大小和数据集的大小，微调可能需要几小时、几天甚至几周的时间。

3. 灾难性遗忘：
   
   当模型学习了一个新的任务后，可能会忘记之前学到的知识。这种现象可能会限制微调模型在实际应用中的通用性和可扩展性。

#### PEFT（Parameter-Efficient Fine-Tuning）

PEFT技术旨在通过最小化微调参数的数量和计算复杂度，来提高预训练模型在新任务上的性能，从而缓解大型预训练模型的训练成本。这样一来，即使计算资源受限，也可以利用预训练模型的知识来迅速适应新任务，实现高效的迁移学习。

<img src="image/PEFT tree.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/> 

图片源自: https://arxiv.org/abs/2312.12148

1. Additive Fine-tuning （Adapter, Prompt Tuning, Prefix-Tuning等）
- 定义：Additive Fine-tuning 指的是在模型的原始架构上增加额外的层或模块，然后只对这些新增的部分进行训练。

2. Partial Fine-tuning
- 定义：Partial Fine-tuning 涉及选择模型中的一部分层（通常是顶层）进行训练，而保持其他层固定。
  
3. Reparameterized Fine-tuning (Lora, QLora等)
- 定义：Reparameterized Fine-tuning 指的是改变模型中参数的表示方式，以优化训练过程。
  
4. Hybrid Fine-tuning
- 定义：Hybrid Fine-tuning 结合了多种微调方法，旨在从不同方法中吸取优点。
  
5. Unified Fine-tuning
- 定义：Unified Fine-tuning提供了一个用于微调的统一框架，它简化了将多种微调方法整合到一个协调的架构中的过程，确保了在模型的适应和优化过程中的一致性和效率。

### 开始微调

这里我们展示一个通过QLoRA去微调Llama2的例子，我们的目标是让Llama2学会写对联

导入依赖库

```
import os                          # 操作系统功能
import torch                       # PyTorch库，用于深度学习
from datasets import load_dataset  # 加载用于训练的数据集
from transformers import (
    AutoModelForCausalLM,          # 用于语言建模任务的AutoModel
    AutoTokenizer,                # 用于令牌化的AutoTokenizer
    BitsAndBytesConfig,           # BitsAndBytes的配置
    HfArgumentParser,             # Hugging Face模型的参数解析器
    TrainingArguments,            # 模型训练的训练参数
    pipeline,                     # 创建模型推理的管道
    logging,                      # 在训练过程中记录信息
)
from peft import LoraConfig, PeftModel  # 用于参数高效微调（PEFT）的包
from trl import SFTTrainer         # 用于监督式微调的SFTTrainer
```

指定基座模型，数据集（如果是本地微调部署则不需要）

```
# Hugging Face训练的基座模型
model_name = "NousResearch/Llama-2-7b-hf"

# 使用的指导数据集
dataset_name = "chenqile09/llama2-chinese-couplet"

# 微调后的模型名称
new_model = "llama-2-7b-miniguanaco"
``

QLora参数配置（刚刚说的模型调整）

```
################################################################################
# QLoRA参数
################################################################################

# LoRA注意力维度
lora_r = 64

# LoRA缩放的Alpha参数
lora_alpha = 16

# LoRA层的Dropout概率
lora_dropout = 0.1
```

模型参数定义

```
################################################################################
# bitsandbytes参数
################################################################################

# 激活4比特精度基础模型加载
use_4bit = True

# 计算4比特基础模型的数据类型
bnb_4bit_compute_dtype = "float16"

# 量化类型（fp4或nf4）
bnb_4bit_quant_type = "nf4"

# 激活4比特基础模型的嵌套量化（双重量化）
use_nested_quant = False

训练基本参数定义

```
################################################################################
# TrainingArguments参数
################################################################################

# 模型预测和检查点将被存储的输出目录
output_dir = "./results"

# 训练周期数
num_train_epochs = 1

# 启用fp16/bf16训练（在A100上设置bf16为True）
fp16 = False
bf16 = False

# 训练时每个GPU的批量大小
per_device_train_batch_size = 4

# 评估时每个GPU的批量大小
per_device_eval_batch_size = 4

# 累积梯度更新步数
gradient_accumulation_steps = 1

# 启用梯度检查点
gradient_checkpointing = True

# 最大梯度正则化（梯度裁剪）
max_grad_norm = 0.3

# 初始学习率（AdamW优化器）
learning_rate = 2e-4

# 应用于除偏置/LayerNorm权重以外的所有层的权重衰减
weight_decay = 0.001

# 使用的优化器
optim = "paged_adamw_32bit"

# 学习率调度（恒定比余弦稍好）
lr_scheduler_type = "constant"

# 训练步数（覆盖num_train_epochs）
max_steps = -1

# 线性预热的步骤比例（从0到学习率）
warmup_ratio = 0.03

# 将具有相同长度的序列分组到批次中
# 节省内存并显著加快训练速度
group_by_length = True

# 每X更新步骤保存一次检查点
save_steps = 25

# 每X更新步骤记录一次
logging_steps = 25


### 上述部分参数的解释
#### **warmup_ratio**：

`warmup_ratio` 参数定义了在训练开始时逐渐增加学习率到设定的最大学习率的过程中，预热阶段所占的训练总步数的比例。

例如，如果 `warmup_ratio` 设置为 0.1，并且总训练步数为 1000 步，那么在前 100 步（1000 * 0.1）中，学习率将从 0 或一个较低的值逐渐增加到设定好的最大学习率。在预热阶段之后，学习率可以按照设定的其他策略（如线性衰减）进行调整。

使用预热策略的目的是帮助模型在训练初期稳定下来，避免一开始使用过大的学习率导致模型训练不稳定或者不收敛。这种策略特别适用于大规模模型和复杂的优化场景，可以帮助模型更好地收敛，提高最终模型的性能和稳定性。

#### **weight_decay**：
`weight_decay` 参数是用来设置权重衰减的，这是一种正则化技术，旨在防止模型过拟合。

权重衰减的工作原理是在模型的损失函数中添加一个额外的项，这个项是模型权重的L2范数（即所有权重值的平方和）乘以一个系数（即 `weight_decay` 参数的值）。

其公式如下：

$$
L = L_0 + \frac{\lambda}{2} (w_1^2 + w_2^2 + ... + w_n^2)
$$

其中 $L_0$ 是模型的原始损失函数，$\lambda$ 是 `weight_decay` 参数的值，$w_1, w_2, ..., w_n$ 是模型的权重值。

更抽象地，上面的式子可以写成下面的公式：

$$
L'(w) = L(w) + \frac{\lambda}{2} ||w||^2
$$

于是，在梯度下降算法更新模型权重时，这会导致梯度更新规则变为：

$$
w_{t+1} = w_t - \alpha (\nabla L(w) + \lambda ||w||)
$$

其中，$\alpha$是学习率，与原来计算新的梯度计算方法相比，多了个$\lambda ||w||$。

从上面的一系列公式和解释，我们可以得到如下结论：

- `weight_decay` 参数的值越大，正则化项对损失函数的影响就越大，从而对模型的权重值进行更强的惩罚。
- 模型的权重值越大，正则化项对损失函数的影响就越大，从而对模型的权重值进行更强的惩罚。
- 若 `weight_decay` 过大，那么原本的损失函数项的影响就会被削弱，从而可能导致模型的参数光顾着变小，而忽略了原本的损失函数项，这样可能会导致模型欠拟合。
                        
因此，我们这样做的效果是在训练过程中对模型的权重进行惩罚，倾向于使权重值更小。权重较小的模型通常具有更好的泛化能力，因为它们依赖于较少的、更强的信号来做出预测，从而减少了对训练数据中**噪声**的拟合。

具体来说，如果 `weight_decay` 设置为一个正数，那么在每次参数更新时，每个权重会被缩小一个固定比例（这个比例由 `weight_decay` 的值决定），这有助于防止权重变得过大，从而有助于防止模型过拟合。如果 `weight_decay` 设置为0，则不会应用权重衰减。

在实际应用中，选择合适的 `weight_decay` 值是一个需要通过实验调整的过程，以找到最佳的模型性能和泛化能力的平衡点。

#### **lr_scheduler_type**：
`lr_scheduler_type` 参数用于指定学习率调度器（Learning Rate Scheduler）的类型。学习率调度器是用来调整训练过程中学习率的策略，以帮助模型更好地收敛或避免过拟合。

`lr_scheduler_type` 参数可以接受以下几种类型的学习率调度器：

1. **Linear**: 学习率会线性地随着训练进度逐渐减小到0。
2. **Cosine**: 学习率遵循一个余弦曲线，从初始学习率减小到0。
3. **CosineWithRestarts**: 这是余弦调度器的一个变种，会在达到最低点后重新开始。
4. **Polynomial**: 学习率按照一个多项式的幂次衰减。
5. **Constant**: 学习率保持不变。
6. **ConstantWithWarmup**: 学习率在开始时通过一个预热期逐渐增加到一个常数值。
7. **LinearWithWarmup**: 学习率在预热期后线性减少。
8. **CosineWithWarmup**: 学习率在预热期后遵循余弦衰减。
9. **CosineWithHardRestarts**: 类似于CosineWithRestarts，但是重启更加突然。

选择合适的学习率调度器可以根据具体的任务和模型架构来优化训练效果。例如，预热期可以帮助模型在训练初期稳定下来，避免因为过高的初始学习率导致的训练不稳定。而逐渐减小的学习率有助于模型在训练后期细化学习，进一步提高性能。

微调参数定义

```
################################################################################
# SFT参数
################################################################################

# 使用的最大序列长度
max_seq_length = None

# 将多个短示例打包在同一输入序列中以提高效率
packing = False

# 将整个模型加载到GPU 0上
device_map = {"": 0}


加载数据集（在第二部分定义的）

```
# 第1步：加载数据集（您可以在此处处理它）
dataset = load_dataset(dataset_name, split="train")

加载配置

```
# 第2步：加载配置
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,                        # 在4比特中加载
    bnb_4bit_quant_type=bnb_4bit_quant_type,      # 4比特量化类型
    bnb_4bit_compute_dtype=compute_dtype,         # 4比特计算数据类型
    bnb_4bit_use_double_quant=use_nested_quant,   # 使用4比特的双重量化
)


检查GPU是否满足训练条件

```
# 第3步：检查GPU对bfloat16的兼容性
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("您的GPU支持bfloat16：使用bf16=True加速训练")
        print("=" * 80)


加载模型

```
# 第4步：加载基础模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,    # 量化配置
    device_map=device_map              # 设备映射
)
model.config.use_cache = False         # 关闭缓存使用
model.config.pretraining_tp = 1        # 预训练的TP参数设置


加载分词器

```
# 第5步：加载LLaMA分词器
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)  # 从预训练模型加载分词器，信任远程代码
tokenizer.add_special_tokens({'pad_token': '[PAD]'})                           # 添加特殊的填充令牌
tokenizer.pad_token = tokenizer.eos_token                                      # 将填充令牌设置为结束令牌
tokenizer.padding_side = "right"                                               # 设置填充方向为右侧


加载LoRA配置

```
# 第6步：加载LoRA配置
peft_config = LoraConfig(
    lora_alpha=lora_alpha,          # LoRA缩放的Alpha参数
    lora_dropout=lora_dropout,      # LoRA层的Dropout概率
    r=lora_r,                       # LoRA注意力维度
    bias="none",                    # 偏置设置
    task_type="CAUSAL_LM",          # 任务类型：因果语言模型
)


设置训练参数

```
# 第7步：设置训练参数
training_arguments = TrainingArguments(
    output_dir=output_dir,                            # 模型预测和检查点将被存储的输出目录
    num_train_epochs=num_train_epochs,                # 训练周期数
    per_device_train_batch_size=per_device_train_batch_size,  # 训练时每个GPU的批量大小
    gradient_accumulation_steps=gradient_accumulation_steps,  # 累积梯度更新步数
    optim=optim,                                      # 使用的优化器
    save_steps=save_steps,                            # 每X更新步骤保存一次检查点
    logging_steps=logging_steps,                      # 每X更新步骤记录一次
    learning_rate=learning_rate,                      # 初始学习率
    weight_decay=weight_decay,                        # 权重衰减
    fp16=fp16,                                        # 启用fp16训练
    bf16=bf16,                                        # 启用bf16训练
    max_grad_norm=max_grad_norm,                      # 最大梯度正则化（梯度裁剪）
    max_steps=max_steps,                              # 训练步数（覆盖num_train_epochs）
    warmup_ratio=warmup_ratio,                        # 线性预热的步骤比例
    group_by_length=group_by_length,                  # 将具有相同长度的序列分组到批次中
    lr_scheduler_type=lr_scheduler_type,              # 学习率调度
    report_to="tensorboard"                           # 报告方式（TensorBoard）
)


设置微调参数

```
# 第8步：设置监督式微调参数
trainer = SFTTrainer(
    model=model,                          # 使用的模型
    train_dataset=dataset,                # 训练数据集
    peft_config=peft_config,              # PEFT（参数高效微调）配置
    dataset_text_field="text",            # 数据集中文本字段的名称
    max_seq_length=max_seq_length,        # 使用的最大序列长度
    tokenizer=tokenizer,                  # 使用的分词器
    args=training_arguments,              # 训练参数
    packing=packing,                      # 是否打包多个短示例以提高效率
)


训练并保存微调模型

```
# 第9步：训练模型
trainer.train()

# 第10步：保存训练后的模型
trainer.model.save_pretrained(new_model)  # 保存经过训练的模型到指定的名称


<img src="image/training.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

### 结果展示

#### 新模型加载

由于PEFT模型一般训练时为了节省存储空间只储存被调整的参数部分，所以加载时需要基础模型与微调模型一起加载

```
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = LlamaTokenizer.from_pretrained(model_name)

<img src="image/result.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

## 三、微调模型的数据准备

本章节内容：
- 数据收集
- 数据预处理
- 数据质量的影响

### 数据收集

#### 语料库/语料
语料库是在自然交流环境中创建的大量有组织的机器可读文本集合。语料库的复数形式是语料。

这些语料可以通过各种方式生成，如电子文本来源、口语转录、光学字符识别等。

相较于较小的语言模型，大型语言模型（LLMs）需要更高质量的数据进行预训练，其模型容量主要取决于预训练语料及其预处理。

#### 数据来源
从各种来源收集大量自然语言语料库对于创建一个熟练的LLM至关重要。目前存在的LLMs主要使用各种公共文本数据集的组合作为它们的预训练语料。

所使用的预训练数据可分为两大类：通用数据和专业数据。

大多数LLMs使用通用数据（如网页、书籍和对话）作为它们的预训练语料，因为这些数据丰富、多样且易于获取。这有助于提高它们的语言建模和泛化能力。

然而，一些研究探索了使用特殊数据集，如多语言数据、科学数据和代码，以赋予LLMs特定的解决问题能力。

**各模型训练数据来源比例**

<img src="image/数据来源.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

图片展示了不同人工智能模型按照它们使用的数据集大小进行比较。数据集大小以GB计量，并且每个模型根据不同类型的数据源分了不同的颜色和柱子。这些数据源包括维基百科、书籍、学术期刊、论坛数据 、Common Crawl以及其他。
“CC”代表“Common Crawl”，这是一个公共的网页数据集，包含了从互联网上抓取的大量网页文本数据。
Common Crawl经常被用来训练大型语言模型，因为它是一个广泛的、多样化的文本来源。

In [7]:
import requests
from bs4 import BeautifulSoup

url = "https://baike.baidu.com/item/%E4%B8%89%E5%8E%9F%E8%89%B2%E5%8E%9F%E7%90%86/6969780?fr=ge_ala"

# 发送请求获取网页内容
response = requests.get(url)

# 检查请求是否成功
if response.status_code == 200:
    # 使用BeautifulSoup解析HTML内容
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # 提取并打印整个网页的文本内容
    # 根据需要，可以修改这部分代码以提取特定元素
    text = soup.get_text()
    print(text)
else:
    print("请求失败，状态码：", response.status_code)


三原色原理_百度百科 网页新闻贴吧知道网盘图片视频地图文库百科百度首页进入词条全站搜索帮助首页秒懂百科特色百科知识专题加入百科百科团队权威合作下载百科APP个人中心三原色原理播报讨论上传视频色度学最基本的原理收藏查看我的收藏0有用+10本词条由“科普中国”科学百科词条编写与应用工作项目 审核 。人眼对红、绿、蓝最为敏感，人的眼睛像一个三色接收器的体系，大多数的颜色可以通过红、绿、蓝三色按照不同的比例合成产生。同样，绝大多数单色光也可以分解成红、绿、蓝三种色光，这是色度学的最基本的原理，也称三原色原理。中文名三原色原理外文名three-primary colours颜    色红、绿、蓝产    生比例合成产生地    位色度学最基本的原理所属领域物理学目录1词语简介2基本内容3配色原理▪原色理论▪混色理论4三原色的应用5相关概念▪色度学▪光谱表色法词语简介播报编辑白光通过棱镜后被分解成多种颜色逐渐过渡的色谱，颜色依次为红、橙、黄、绿、蓝、靛、紫 [4]，这就是可见光谱。其中人眼对红、绿、蓝最为敏感，人的眼睛就像一个三色接收器的体系，大多数的颜色可以通过红、绿、蓝三色按照不同的比例合成产生。同样绝大多数单色光也可以恋享愚分解成红绿蓝三种色光。这是色度学的最基本原理，即三基色原理。三种基色是相互独立的，任何一种基色都不能有其它两种颜色合成。红绿蓝是三基色，这三种颜色合成的颜色范围最为广泛。红绿蓝三基色按照不同的比例相加合成混色称为相加混色。红色+绿雅旬店色=黄色绿色+她夜糠微酷踏蓝色=青色红色炼备背+蓝色=亮誉酷断紫色红色+绿色归她+蓝色=整探墓白色基本内容播报编辑彩色全息显示实际上是一种三维彩色图像传递技术。 根据色度学原理，利用红光、 绿光和蓝光经过适当比例混合后，可产生自然界大部分的颜色，因此把红光、 绿光和蓝光称作三原色。 因此彩色全息图的颜色记录和再现实质上是三原色的记录和再现。任何光都可以用红、绿、蓝这3种光按不同的比例混合而成，这就是三原色原理。三原色的原理可解释如下：（1）自然界的任何光色都可以由3种光色按不同的比例混合而成。（2）三原色之间是相互独立的，任何一种光色都不能由其余的两种光色来组成。（3）混合色的饱和度由3种光色的比例来决定。混合色的亮度为3种光色的亮度之和。配色原理播报编辑原色理论三原色，所谓三原色，就是指这三种色中的任意一色都不能由另外

### 数据预处理

为LLMs创建预训练语料库时，对收集到的文本数据进行预处理以去除噪音、冗余、不相关和潜在有害内容是非常重要的。

这是因为数据质量会显著影响语言模型的能力和性能。本节讨论了各种数据预处理策略，以提高收集数据的质量。

#### 数据标注 

In [8]:
import os
from dotenv import load_dotenv
load_dotenv('../.env')  # 从上级目录的.env文件加载环境变量
from openai import OpenAI
from functools import lru_cache

class BaseLLM:
    @lru_cache(maxsize=1024)  # 使用LRU缓存来存储最近使用的请求结果
    def chat(self, text):
        return self._chat(text)
    
    def _chat(self, text):
        raise NotImplementedError  # 如果直接调用_base类的_chat方法，则抛出未实现错误

class OpenAILLM(BaseLLM):
    def __init__(self, model_name):
        self.client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),  # 从环境变量获取API密钥
            base_url=os.getenv("OPENAI_API_BASE")  # 从环境变量获取API基础URL
        )
        self.model_name = model_name  # 设定模型名称
        self.conversation_history = []  # 初始化对话历史列表

    def convert_text_to_prompt(self, instr, target):
        return instr.format(target)  # 格式化指令文本

    def __call__(self, text):
        return self.chat(text)  # 允许对象像函数一样被调用，内部转发到chat方法
        
    def chat(self, text, temperature=0, messages=[], stops=None):
        return self._chat(text, temperature, messages, stops)
        
    def _chat(self, text, temperature, messages=[], stops=None):
        if not messages:
            messages = [{"role": "user", "content": text}]  # 如果没有提供消息，创建一个用户角色的消息
        print("开始请求模型%s" % (self.model_name))  # 打印日志，表示开始请求模型
        
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            stream=False,
            stop=stops,
            temperature=temperature,
        )
        
        return response.choices[0].message.content  # 返回模型的回答
        
    def history_chat(self, text, message=[], stops=None):
        self.conversation_history.append({"role": "user", "content": text})  # 添加用户消息到对话历史

        # 创建请求
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=self.conversation_history,  # 传递完整的对话历史
            stream=False,
        )

        # 将模型的回答添加到对话历史中
        answer = response.choices[0].message.content
        self.conversation_history.append({"role": "assistant", "content": answer})
        return answer

gpt_3_5_turbo = OpenAILLM("gpt-3.5-turbo")  # 创建一个使用gpt-3.5-turbo模型的对象


In [9]:
print(gpt_3_5_turbo.chat("以json格式梳理下面的文本，请将下面的文本分成{instruction: 问题,input:不填写 ,output: 答案}的格式，并且将多个问题答案对放入列表中" + text))

开始请求模型gpt-3.5-turbo
[
  {
    "instruction": "三原色原理是什么？",
    "input": "",
    "output": "三原色原理是指人眼对红、绿、蓝最为敏感，大多数的颜色可以通过红、绿、蓝三色按照不同的比例合成产生。"
  },
  {
    "instruction": "三原色的中文名和外文名分别是什么？",
    "input": "",
    "output": "三原色的中文名是三原色原理，外文名是three-primary colours。"
  },
  {
    "instruction": "三原色的颜色是什么？",
    "input": "",
    "output": "三原色的颜色是红、绿、蓝。"
  },
  {
    "instruction": "三原色的应用有哪些？",
    "input": "",
    "output": "三原色的应用包括彩色全息显示和绘画、印刷领域。"
  },
  {
    "instruction": "什么是色度学？",
    "input": "",
    "output": "色度学是研究颜色度量和评价方法的一门学科，是颜色科学领域的一个重要部分。"
  },
  {
    "instruction": "什么是光谱表色法？",
    "input": "",
    "output": "光谱表色法是一种以分光光度曲线来表示颜色特性的方法，用于描述非发光物体和发光体的颜色。"
  }
]


#### 质量过滤

确保训练语料库只包含高质量数据的两种主要方法：**基于分类器的方法和基于启发式的方法**。

基于分类器的方法训练一个二元分类器来识别和过滤低质量数据，以高质量文本（例如，维基百科页面）作为正例，候选数据作为负例。

然而，基于分类器的方法可能无意中移除了方言、口语和社会方言等高质量文本，导致偏见和语料库多样性的减少。

相比之下，有些**启发式的方法采用精心设计的规则**来排除低质量文本。

这些规则包括基于语言的过滤、基于度量的过滤、基于统计的过滤和基于关键字的过滤。

基于语言的过滤删除与LLM任务无关的语言文本，而基于度量的过滤使用评估度量，如**困惑度**来检测不自然的句子。基于统计的过滤根据语料库的统计特征来衡量文本质量，而基于关键字的过滤则根据特定的关键字集（如HTML标签、超链接、样板文本和冒犯性词汇）来移除噪声或无用元素。

**困惑度**定义为语言模型对测试集的概率的倒数的几何平均。具体来说，对于一个长度为 \(N\) 的句子 \(w_1, w_2, ..., w_N\)，其困惑度的计算公式为：

$$
PP(W) = P(w_1, w_2, ..., w_N)^{-\frac{1}{N}}
$$

其中 $$P(w_1, w_2, ..., w_N) = P(w_1) \times P(w_2|w_1) \times ... \times P(w_N|w_1, w_2, ..., w_{N-1})$$

它是模型赋予该句子的概率。如果模型对句子中的词序列预测得越准确，这个概率就越高，从而困惑度就越低。

困惑度可以用来检测不自然的句子，因为不自然的句子（例如语法错误的句子）通常会有更高的困惑度。这是因为这些句子的结构或用词可能与训练模型时使用的自然语言文本有显著差异。

#### 示例：

假设我们有一个训练好的语言模型（如已经完成预训练的GPT-2），我们可以利用它来评估以下两个句子的困惑度：

1. 正常的句子："我今天"
2. 不自然的句子："我天今"

在中文中，句子2比句子1显得不自然。对于这个例子，GPT-2会针对每个词输出不同的概率分布，我们可以根据这个概率分布依照上面的公式计算句子的困惑度。

例如，对于第一个句子

$$ P(我，今，天) = P(我) \times P(今|我) \times P(天|我，今) $$

对于第二个句子

$$ P(我，天，今) = P(我) \times P(天|我) \times P(今|我，天) $$

显然，第二个句子的困惑度可能会更高，因为‘我天’后面接‘今’的概率可能会比较低。

#### 去重复

以往的研究表明，在预训练语料库中存在**重复数据会减少语言模型的多样性**，可能导致训练过程中的不稳定，并对模型的性能产生负面影响。

因此，从语料库中移除重复句子是必要的。这可以在不同的层面上进行，包括句子级、文档级和数据集级。

在句子级别，包含重复短语或词汇的低质量句子应该被移除，以避免将重复模式引入模型中。

在文档级别，先前的研究依赖于表面特征重叠（例如，词汇和n-gram）来识别和移除具有相似内容的重复文档。

此外，为了防止训练集和评估集之间的污染，重要的是要从训练集中移除可能的重复文本。


#### 隐私编辑

用于预训练大型语言模型（LLMs）的大部分文本数据来自网络，经常包含用户生成的内容，可能涉及敏感或个人信息。

这带来了隐私泄露的风险，因此有必要从预训练语料库中移除**个人可识别信息（PII）**。

基于规则的方法，如关键字检测，可以识别并移除姓名、地址和电话号码等PII。此外，预训练语料库中重复的PII数据可能使LLMs容易受到隐私攻击。

因此，去重复也有助于降低隐私风险。

### 数据质量的影响

反复训练LLMs是不可行的，因此，在训练LLM之前拥有一个高质量、准备充分的预训练语料库至关重要。在本节中，我们将探讨预训练语料库的质量和分布如何影响LLMs的性能。

重复数据可能导致“双重下降”或压倒训练过程，并且会降低LLMs从上下文中复制的能力，可能影响其泛化能力。

“双重下降”这个术语描述了一个机器学习现象，即模型的泛化误差随着模型复杂度的增加最初会降低，达到一个最小值，然后随着模型变得更复杂而再次上升。

与普遍的看法相反，过拟合应该导致模型的泛化误差随着模型复杂度的不断增加而增加。

第一次下降代表了传统的偏差与方差之间的权衡，即模型复杂度的增加会因为偏差的下降而导致对训练数据的更好拟合，但同时也会因为方差的增加而导致对测试数据的拟合变差。

插值阈值是偏差和方差曲线汇合的地点。当模型复杂度超过这个点时，泛化误差又会降低。

这种第二次下降可能是由于模型的正则化特性所引起的，这些特性可以防止**过拟合**并提高泛化性能。

对低质量数据（如噪声、有害或重复数据）进行预训练会损害语言模型的性能。

为了开发有效的LLM，考虑收集到的训练数据的数量和质量都很重要。包括T5、GLaM和Gopher在内的最新研究已经调查了数据质量对下游任务性能的影响。

比较在过滤和未过滤数据上训练的模型发现，对清洁数据进行预训练可以提高性能。

## 四、微调在模型训练中的位置

<img src="image/pipeline.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

### 大模型训练流程

**预训练**
<img src="image/预训练流程.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

- 从互联网上收集一个大型且多样化的数据集。这个数据集包含了来自广泛来源的文本，以确保模型学习到各种语言模式。
- 清洗和预处理数据，去除噪声、格式问题以及不相关信息。
- 将清洗后的文本数据进行分词处理，分解成更小的单元，如单词或子词单元（例如，字节对编码或WordPiece）。
- 对于像GPT-3这样的大型语言模型（LLM），通常使用Transformer架构，因为它们在处理序列数据方面非常有效。
- 大型语言模型（LLM）的预训练是通过训练模型预测文本序列中的下一个单词进行的，它使用了大量的数据集，使其能够理解和生成类似人类的语言。

**预训练的输出**
如果我们在仅仅完成预训练之后就使用模型，此时它基于输入的数据学会了如何“正确地”预测下一个单词，但我们可以注意到，在这时模型有一定概率会输出无意义的值，并且不会在正确的点停下来。
<img src="image/预训练输出例子.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

**SFT(Supervised fine-tuning) 或者 instruction tuning**
- 在这个过程中，模型将用户的信息作为输入，以AI训练者的回应作为目标。模型通过最小化其预测和提供的回应之间的差异来学习生成回应。
- 在这一阶段，模型能够理解指令的含义以及如何根据提供的指令从其记忆中检索知识。

<img src="image/SFT流程.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

**微调结果展示**

<img src="image/SFT输出例子.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

**Reward model**

<img src="image/ChatGPT应用大模型微调技术.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

**RLHF(Reinforcement Learning from Human Feedback)**

<img src="image/why rl.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

我们将RHFL作为第二步微调，进一步根据我们讨论过的标准对模型进行校准：有益、诚实和无害。

RHFL的目标是：
最大化有益性，
最小化伤害，
避免危险话题。

我们不会深入探讨强化学习的工作细节（可以讨论），总而言之，我们可以通过训练神经网络（NN）模型与环境互动，通过连续的决策来最大化累积的奖励信号。

在RLHF中，我们会为同一个提示生成多个输出，并请测试人员对输出从最佳到最差进行排序。这些数据用于训练另一个称为奖励模型的神经网络模型。这个奖励模型现在能够理解人类偏好。可以将其视为由专家训练实习生，以识别有益、诚实和无害的内容。

<img src="image/reward.png"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

一旦奖励模型训练完成，它就可以代替人类来标记数据，基于这些反馈进一步在大规模上微调大型语言模型（LLM）。

### 微调技术与提示工程（在未微调的模型上）的特性对比

| 特性 | 提示工程 | 微调 |
| ---- | -------- | ---- |
| 可接受数据量 | 无法输入大量的数据，token有限制 | 理论上可以输入无限量的数据进行微调 |
| 幻觉问题 | 模型幻觉难以纠正 | 纠正模型存在的错误信息 |
| 面对大量数据的表现 | 如果输入大量数据，模型可能会遗忘 | 模型可以从大量数据中进行学习新的信息 |
| 成本 | 使用成本低 | 微调需要大量算力成本 |
| 使用门槛 | 使用门槛低 | 需要特定的技术能力 |
| 使用场景 | 通用场景，快速启动项目和产品原型 | 行业应用，企业级应用 |


## 五、PEFT方法对比 

本章节内容：
- Prefix Tuning 论文：[Prefix-Tuning: Optimizing Continuous Prompts for Generation](https://arxiv.org/pdf/2101.00190.pdf)
- LoRA 论文：[LORA: LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS](https://arxiv.org/pdf/2106.09685.pdf)
- QLoRA 论文：[QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/pdf/2305.14314.pdf)


扩展阅读：
- P-Tuning v1 论文：[GPT Understands, Too](https://arxiv.org/pdf/2103.10385.pdf)
- P-Tuning v2 论文：[P-Tuning: Prompt Tuning Can Be 
Comparable to Fine-tuning Across Scales and Task](https://aclanthology.org/2022.acl-short.8.pdf))

### [Adapter](https://arxiv.org/pdf/1902.00751.pdf)

<img src="image/adapter.png"  width="400" height="400" alt="rag-vs-finetuning-chn"/>

- Adapter位于Feed-Forward Layer之后、**残差连接**之前。Adapter本质上就是两层**MLP**，分别负责将Transformer的表征降维和升维（右图）。

- 基于Adapter的方法， 只需要添加不到5%的可训练参数，就可以几乎达到全参数训练的效果 ，在训练过程中大大节省了训练时间，做到时间有效性。因此在真实场景应用时，不同的任务我们不需要重新对整个预训练模型进行微调，我们只需要保存Adapter即可 ，而预训练模型的其他参数都是原始预训练的，这样就做到了空间的有效性。

#### 残差连接简介
残差网络（ResNet）是一种深度神经网络架构，最初由微软研究院的Kaiming He等人在2015年提出。它通过引入“残差学习”的概念来解决深度网络中的退化问题，即随着网络层数的增加，网络性能反而下降的问题。

残差网络的核心原理是引入了“残差链接”（Residual Connection），即将输入直接加到输出上，从而使网络可以学习输入与输出之间的残差。这种结构使得网络可以更容易地学习恒等映射，从而**缓解**了深度网络训练中的梯度消失问题。

在传统的深度神经网络中，每一层都直接学习输入到输出的映射。而在残差网络中，每个单元会学习输入和输出之间的残差，即差异部分。具体来说，如果我们期望从层的输入x学到的目标映射为F(x)，残差网络不是直接学习这个映射，而是学习目标映射与输入之间的残差F(x) - x。这样，网络的输出就仍然等效于目标映射F(x)。

<div style="text-align:center">
    <img src="./image/残差链接.jpg" alt="替代文本" width="764" height="255">
</div>

理论上，这样做能够简化与加速学习过程，因为学习残差往往比学习原始映射更容易。

总的来说，残差网络通过引入跳跃连接，使得网络学习的输入与目标输出的差，从而有效解决了深度网络训练中的退化问题。你可以在论文[《Deep Residual Learning for Image Recognition》](https://arxiv.org/abs/1512.03385)中进一步了解残差网络的原理。


#### 多层感知机（MLP）简介
多层感知机（Multilayer Perceptron，MLP）是一种最基本的前馈神经网络，由**多个全连接层**组成。每个全连接层由多个神经元组成，每个神经元与上一层的所有神经元相连。MLP的核心是通过多个全连接层来学习输入数据的非线性映射。

<div style="text-align:center">
    <img src="./image/MLP1.png" alt="替代文本" width="600">
</div>

从上图可以看出，每个全连接层就是一个线性变换（与权重W的矩阵乘法和偏置b矩阵加法）和一个非线性激活函数（如ReLU）的组合。这种结构使得MLP能够学习输入数据的复杂模式，从而适应各种不同的任务。

总之，MLP通过非线性映射去学习输入与输出之间的复杂的变化模式，是一种最为基本的前馈神经网络。

### Prefix Tuning

<img src="image/prefix1.jpg"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

<img src="image/prefix2.jpg"  width="500" height="600" alt="rag-vs-finetuning-chn"/>

1. **提出原因**：为了减轻对于大型预训练语言模型进行全量微调的资源需求，Prefix-Tuning提供了一种轻量化的替代方案，减少了每个任务需要存储的模型副本。

Prefix-tuning 是一种自然语言处理（NLP）领域的模型微调技术。这种技术的提出主要是为了解决以下问题：

**参数效率低下**：传统的微调方法需要更新预训练模型的全部参数，这对于参数量巨大的模型来说既不经济也不高效。对于需要快速适应多个任务的应用场景，全参数微调需要大量的存储空间和计算资源。

**迁移学习的灵活性有限**：预训练模型经过特定任务的全参数微调后，往往难以再次适应新的任务，因为原始预训练时学习到的知识可能被覆盖。

**灾难性遗忘**：在微调过程中，模型可能会丢失在预训练阶段学到的一般性知识，这种现象被称为灾难性遗忘。这会导致模型无法保持在多个任务上的表现。

**缓慢的部署和迭代**：全参数微调要求对每个新任务重新训练整个模型，这使得模型部署和迭代变得缓慢。

如果不解决这些问题，可能会有以下严重的危害：

**计算资源的浪费**：由于每个任务都需要全参数微调，使用大型预训练模型将变得不可持续，特别是在资源有限的环境中。

**部署困难**：对于需要快速适应和更新的现实世界应用，模型的部署会因为微调的计算成本而变得十分困难。

**模型性能下降**：灾难性遗忘会降低模型在多任务学习环境中的性能，影响模型在现实世界问题中的应用效果。

**研究和应用的局限性**：对于研究者来说，没有一个高效的微调方法将限制模型的探索和创新。对于企业来说，缓慢的迭代周期会减缓产品的上市速度和市场竞争力。

因此，Prefix-tuning 提出了一种优化少量参数的方法，通过在模型输入前加入一小段“前缀”序列来指导模型生成特定任务的响应，而不需要改变模型的大部分参数。


2. **一般流程**：该方法优化一系列连续的、特定任务的向量序列（即前缀），这些前缀作为“虚拟tokens”加在输入之前，从而影响后续token的生成，而无需改变底层的模型参数。

   Prefix-Tuning的流程可以更清晰地描述如下：

 ### Prefix-tuning的一般实现步骤：

**选择基础模型**：选定一个已经预训练好的语言模型，如BERT、GPT-2等，作为微调的起点。

**定义前缀**：为每个任务或每种输入类型创建一个可学习的前缀序列。这些前缀是一组连续的向量（或token），其长度远小于输入序列的长度。

**集成前缀到模型中**：将前缀序列集成到模型的输入层或注意力层中。在输入层中，前缀可以直接添加到输入序列的开始；在注意力层中，前缀可以作为额外的键（K）和值（V）被注入到自注意力机制中。

**微调训练**：在训练过程中，只有**前缀序列的参数和可能的少量模型参数**会被更新，模型的其他部分保持固定。训练数据将通过模型前向传播，同时携带前缀序列。

**正则化和约束**：为了保持模型性能和避免过拟合，可以应用正则化方法，如dropout、权重衰减等，也可以对前缀参数的更新量设定约束。

**评估和迭代**：使用验证集评估模型的性能，并根据需要对前缀参数和模型结构进行迭代调整。

**部署**：完成微调后的模型可以部署到特定的任务中，用于推断新的输入数据。


这个过程的关键在于前缀的设置和训练，它允许模型适应新任务，而无需重训练或改变大量的模型参数。


图片展示了前缀长度与性能之间的关系。图表显示了随着前缀长度的增加，性能也在增加，这表明更长的前缀可以提供更多的表达能力，从而改善生成结果。同时，图表还指出了在训练和推断速度方面，更长的前缀对每个批次的影响可以忽略不计，因为GPU上的注意力计算是并行化的。

<img src="image/pl.jpg"  width="800" height="600" alt="prefix_length_Performance"/>

3. **方法样例**：在GPT-2上进行表格到文本的生成和在BART上进行摘要生成，表现与全量微调相当，但仅修改了0.1%的参数。


4. **存在问题**：在某些任务上与全量微调相比有性能下降。平衡前缀长度和表达能力与过拟合的问题需要仔细调整。对于前缀调优如何改善外推能力以及在不同应用场景的潜在局限性，还需要进一步研究。

Prefix-Tuning的核心思想，强调了它在优化语言模型性能的同时，解决了计算效率和存储限制的问题。

### 实现Prefix-tuning的具体编码示例（伪代码）：

In [ ]:
```
# 假设使用PyTorch和Huggingface的Transformers库
from transformers import GPT2Model, GPT2Config
import torch

# 加载预训练模型
model = GPT2Model.from_pretrained('gpt2')

# 定义前缀
prefix_length = 10  # 前缀的长度
num_parameters = model.config.n_embd  # 每个前缀向量的维度
prefix = torch.nn.Parameter(torch.zeros(1, prefix_length, num_parameters))

# 微调过程
optimizer = torch.optim.Adam([prefix] + [p for p in model.parameters() if p.requires_grad], lr=1e-5)
model.train()

for epoch in range(num_epochs):
    for batch in data_loader:
        inputs, labels = batch
        # 将前缀添加到输入数据中
        inputs_with_prefix = torch.cat((prefix.expand(inputs.size(0), -1, -1), inputs), dim=1)
        outputs = model(inputs_with_prefix, labels=labels)
        loss = outputs.loss
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


huggingface peft关于prefix-tuning的核心代码实现在[prefix_tuning](https://github.com/huggingface/peft/blob/main/src/peft/tuners/prefix_tuning/model.py)

In [ ]:
```
class PrefixEncoder(torch.nn.Module):
    def __init__(self, config):
        super().__init__()
        # 是否使用前缀投影，一种在处理前缀时使用的特定技术
        self.prefix_projection = config.prefix_projection
        # 嵌入向量的维度
        token_dim = config.token_dim
        # 模型中的层数
        num_layers = config.num_layers
        # 编码器隐藏层的大小
        encoder_hidden_size = config.encoder_hidden_size
        # 虚拟（或者说“伪”）令牌的数量
        num_virtual_tokens = config.num_virtual_tokens
        if self.prefix_projection and not config.inference_mode:
            # 如果使用前缀投影且不处于推理模式，则使用一个两层的多层感知机（MLP）来编码前缀
            self.embedding = torch.nn.Embedding(num_virtual_tokens, token_dim)
            self.transform = torch.nn.Sequential(
                torch.nn.Linear(token_dim, encoder_hidden_size),  # 从令牌维度到隐藏层大小的线性变换
                torch.nn.Tanh(),  # 激活函数
                torch.nn.Linear(encoder_hidden_size, num_layers * 2 * token_dim),  # 从隐藏层大小到最终输出维度的线性变换
            )
        else:
            # 如果不使用前缀投影，或者处于推理模式，直接使用嵌入层映射虚拟令牌
            self.embedding = torch.nn.Embedding(num_virtual_tokens, num_layers * 2 * token_dim)

    def forward(self, prefix: torch.Tensor):
        # 前向传播函数，根据是否使用前缀投影来处理输入的前缀
        if self.prefix_projection:
            prefix_tokens = self.embedding(prefix)  # 获取前缀令牌的嵌入表示
            past_key_values = self.transform(prefix_tokens)  # 通过MLP变换得到最终的键值对
        else:
            past_key_values = self.embedding(prefix)  # 直接


### LoRA

<img src="image/LoRA.jpg"  width="500" height="600" alt="rag-vs-finetuning-chn"/>

1. **提出原因**：为了在大型预训练语言模型（如GPT-3）上进行更高效的微调，同时减少存储和计算成本。

2. **解决问题**：
   - 减少大型模型微调所需的资源消耗，特别是减少训练时的GPU内存需求。
   - 允许在保持原有预训练模型权重不变的情况下，通过微调少量参数来适应下游任务。
   - 降低部署不同微调模型实例的成本，尤其是在模型规模非常大时。
  
   如果不解决高效微调的问题，那么在需要快速适应的实际应用中部署大型模型将变得困难。这将导致成本增加，限制了资源有限用户的使用，并可能减缓大型语言模型的研究和应用步伐。


   
与Prefix tuning的区别：
更新的参数部分：LoRA主要更新的是模型内部的权重矩阵，而Prefix tuning更新的是模型输入的一部分。
方法的侵入性：LoRA对模型的侵入性较小，因为它通过小规模的权重更新实现调整，而Prefix tuning则通过改变模型的输入实现调整。
适用性：虽然两者都能实现任务的快速切换，但在实现细节和适应特定任务的方式上存在差异。

3. **一般流程**：
    - 模型选择和权重冻结：选择一个大型预训练语言模型（如GPT-3），并冻结它的所有原始权重。    
    - 低秩分解矩阵的引入：在模型的每个层中引入两个低秩矩阵（记为A和B）。这些矩阵将与原始权重矩阵结合，形成更新后的权重。
    - 优化低秩矩阵：在微调过程中，只对这些低秩矩阵的参数进行优化，而非整个模型的所有参数。
    - 训练和应用：在特定下游任务的数据上训练这些参数。训练完成后，将训练好的低秩矩阵与原始权重结合，用于特定任务的预测或生成。
    - 任务切换：在需要切换到不同的下游任务时，可以通过更换不同的低秩矩阵来实现，这个过程快速且内存开销小。


4. **方法的样例**：
   - 在GPT-3模型上应用LoRA进行自然语言理解（NLU）和自然语言生成（NLG）任务。
   - 在保持原模型结构不变的情况下，只优化低秩矩阵参数来适应不同任务。

5. **存在的问题**：
   - LoRA的最优秩的确定以及如何适应特定任务的权重选择还需要进一步的研究和实验验证。


LoRA的未来研究方向包括将其与其他高效的适应方法结合起来，以获得更大的改进，探索微调或LoRA成功背后的机制，并研究更原理化的权重选择方法进行适应。同时，也有兴趣了解LoRA更新中观察到的秩亏缺性是否暗示了原始预训练模型中类似的属性。


<img src="image/lora.png"  width="600" alt="rag-vs-finetuning-chn"/>




[1,2,3]
[3,4,5]
[5,6,7]


3*1 1*3

LoRA层定义

In [ ]:
class LoRALayer():
    def __init__(
        self, 
        r: int,  # 'r' 是 LoRA 层的秩，用于控制 LoRA 层的参数数量
        lora_alpha: int,  # 'lora_alpha' 是 LoRA 层的学习率缩放因子，用于调整学习速率
        lora_dropout: float,  # 'lora_dropout' 是应用于 LoRA 层的dropout比例，用于正则化和减少过拟合
        merge_weights: bool,  # 'merge_weights' 指示是否合并 LoRA 层的权重到原始模型权重
    ):
        self.r = r
        self.lora_alpha = lora_alpha
        # 如果 lora_dropout 大于0，则使用指定的 dropout 比例，否则不应用 dropout
        if lora_dropout > 0.:
            self.lora_dropout = nn.Dropout(p=lora_dropout)
        else:
            self.lora_dropout = lambda x: x  # 如果 dropout 比例为 0，则定义一个空操作
        # 将权重标记为未合并
        self.merged = False
        self.merge_weights = merge_weights

必要参数定义

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class Linear(nn.Linear, LoRALayer):
    # 在一个Dense layer中实现LoRA
    def __init__(
        self, 
        in_features: int,  # 输入特征数
        out_features: int,  # 输出特征数
        r: int = 0,  # LoRA 参数 r，控制权重矩阵的秩
        lora_alpha: int = 1,  # LoRA 参数 alpha，用于缩放权重
        lora_dropout: float = 0.,  # LoRA 参数 dropout，用于权重的可选丢弃
        fan_in_fan_out: bool = False,  # 如果要替换的层存储权重类似于 (fan_in, fan_out)，则将其设置为 True
        merge_weights: bool = True,  # 是否合并权重
        **kwargs
    ):
        nn.Linear.__init__(self, in_features, out_features, **kwargs)
        LoRALayer.__init__(self, r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout,
                           merge_weights=merge_weights)

        self.fan_in_fan_out = fan_in_fan_out
        # 实际可训练参数
        if r > 0:
            self.lora_A = nn.Parameter(self.weight.new_zeros((r, in_features)))  # LoRA 参数 A
            self.lora_B = nn.Parameter(self.weight.new_zeros((out_features, r)))  # LoRA 参数 B
            self.scaling = self.lora_alpha / self.r  # 缩放因子
            # 冻结预训练的权重矩阵
            self.weight.requires_grad = False
        self.reset_parameters()
        if fan_in_fan_out:
            self.weight.data = self.weight.data.transpose(0, 1)

    def reset_parameters(self):
        nn.Linear.reset_parameters(self)
        if hasattr(self, 'lora_A'):
            # 初始化 B 与 nn.Linear 默认方式相同，A 初始化为零
            # 这与论文中描述的不同，但不应影响性能
            nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
            nn.init.zeros_(self.lora_B)

    def train(self, mode: bool = True):
        '''切换训练模式'''
        def T(w):
            return w.transpose(0, 1) if self.fan_in_fan_out else w
        nn.Linear.train(self, mode)
        
        # 权重合并时的逻辑
        if mode:
            if self.merge_weights and self.merged:
                # 执行合并
                if self.r > 0:
                    self.weight.data -= T(self.lora_B @ self.lora_A) * self.scaling
                # 确保训练时权重不再合并
                self.merged = False
        else:
            if self.merge_weights and not self.merged:
                # 执行权重合并并标记
                if self.r > 0:
                    self.weight.data += T(self.lora_B @ self.lora_A) * self.scaling
                self.merged = True       

    def forward(self, x: torch.Tensor):
        def T(w):
            return w.transpose(0, 1) if self.fan_in_fan_out else w
        if self.r > 0 and not self.merged:
            result = F.linear(x, T(self.weight), bias=self.bias)            
            result += (self.lora_dropout(x) @ self.lora_A.transpose(0, 1) @ self.lora_B.transpose(0, 1)) * self.scaling
            return result
        else:
            return F.linear(x, T(self.weight), bias=self.bias)


### QLoRA

#### 什么是模型量化

- 模型量化的定义：模型量化是指将神经网络中的权重和激活从浮点数（如32位浮点数）转换为定点数（如8位整数）。这个过程通常用于优化模型的大小和运算速度。

- 常规精度模型：这通常指使用32位浮点数（FP32）来表示模型的权重和激活，这是许多神经网络默认的数值格式。

- 低精度模型：在这里，低精度指的是使用比标准32位浮点数（FP32）更低的数值精度来表示模型的权重和激活。常见的格式包括16位浮点数（FP16，也称为半精度浮点）和8位定点整数（INT8）。目前，低精度通常指的是INT8，这是因为使用8位整数可以进一步减少模型的大小和计算需求。

- 混合精度：混合精度训练是一种同时使用FP32和FP16的方法。在这种设置中，某些部分的模型使用FP16以减少内存使用和提高计算速度，而其他关键部分仍然使用FP32以保持计算的准确性。FP16减少了一半的内存需求，但是并不是所有的操作和参数都适合用FP16表示，因为这可能会导致精度损失。

#### 什么是QLoRA

<img src="image/QLoRA.jpg"  width="800" height="600" alt="rag-vs-finetuning-chn"/>

**提出原因**：由于微调大型语言模型（如65亿参数模型）需要巨大的计算资源，对于许多研究者和实践者来说不可行。因此，QLoRA被提出，旨在大幅降低内存使用，使得在单个GPU上微调这类模型成为可能。

QLoRA (Quantized Low-Rank Adaptation) 是一种高效的微调方法，旨在降低大型语言模型微调时的内存使用量，使得一个具有650亿参数的模型能够在单个48GB GPU上进行微调，同时保持完整的16位微调任务性能。QLoRA通过一个冻结的4比特量化预训练语言模型传播梯度到低秩适配器（LoRA）。此方法引入了多种创新以节省内存，包括一个新的数据类型4比特NormalFloat、双重量化以减少平均内存占用，以及页面优化器来管理内存峰值。

QLoRA提出的原因是，尽管量化方法可以减少大型语言模型的内存占用，但这些技术仅适用于推理，并在训练期间失效。QLoRA解决了这一问题，首次实现了在不降低性能的情况下对量化模型进行微调。这一方法不仅解决了现有微调方法在内存需求上的局限性，而且还提高了模型训练的可达性和效率。

如果不解决这一问题，将会导致在有限的硬件资源下无法有效微调大型语言模型，这将限制模型性能的提升以及其在特定任务上的适应性。在法律、医疗等对模型准确性要求极高的领域，这一限制可能会严重影响模型的实用性和可信度。因此，QLoRA的开发对推动大型语言模型的实用化和普及化具有重要意义。


**解决问题**：QLoRA（Quantized Low-Rank Adaptation）和LoRA（Low-Rank Adaptation）都是为了提升大型预训练语言模型在特定任务上的性能和适应性，同时减少微调过程中的资源需求。LoRA通过在模型的权重矩阵中引入低秩结构来实现这一点，这样可以减少需要更新的参数数量，从而降低内存和计算成本。而QLoRA在LoRA的基础上进一步引入了量化技术，允许在更低的位宽（如4比特）下对模型进行微调，从而大幅减少了模型训练时的内存占用。

不同之处在于，LoRA主要关注通过低秩结构提高参数效率，而QLoRA则在此基础上进一步通过量化技术，减少了存储和操作参数时所需的内存和计算资源。QLoRA的优势在于使得在资源受限的设备上微调大型模型成为可能，而不会牺牲模型的性能。


**存在问题**：在特定场景或任务中，QLoRA的量化和优化技术可能不够有效；在内存效率和模型性能之间的平衡可能需要针对特定应用进行微调；该方法可能在捕捉非常大数据集或极其复杂任务的细微之处上有限制。

经过分析QLORA的论文，可以概括出它存在的一系列问题：

- **数据质量 vs 数据量**：QLORA的实验结果表明，数据质量比数据量更重要。例如，较小但高质量的数据集在聊天机器人性能上优于大型数据集。

- **性能与精确度的平衡**：QLORA使用4位精度进行微调，虽然在某些方面能够匹配16位全模型微调的性能，但在特定情况下仍存在性能损失。

- **模型输出可靠性**：在某些情况下，QLORA生成的回答可能在表面上看起来自信，但实际上是不准确的。

- **抗错误信息的能力**：QLORA在某些情况下显示出对错误信息的抵抗力，但这可能取决于特定的情境和问题。

#### 方法调用

SFT Trainer：https://huggingface.co/docs/trl/sft_trainer

In [ ]:
# LoRA注意力维度
lora_r = 64

# LoRA缩放的Alpha参数
lora_alpha = 16

# LoRA层的丢弃概率
lora_dropout = 0.1

# 加载LoRA配置
peft_config = LoraConfig(
    lora_alpha=lora_alpha,  # LoRA的缩放系数Alpha
    lora_dropout=lora_dropout,  # LoRA层的丢弃概率
    r=lora_r,  # LoRA的注意力维度
    bias="none",  # 偏置配置
    task_type="CAUSAL_LM",  # 任务类型为因果语言模型
)

# 从预训练加载模型，并配置设备映射
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map  # 设备映射配置
)

# 创建SFT（Structured Fine-Tuning）训练器
trainer = SFTTrainer(
    model=model,  # 指定模型
    train_dataset=dataset,  # 训练数据集
    peft_config=peft_config,  # PEFT（Parameter-Efficient Fine-Tuning）配置，此处为LoRA配置
    dataset_text_field="text",  # 数据集中文本字段的名称
    max_seq_length=max_seq_length,  # 最大序列长度
    tokenizer=tokenizer,  # 用于文本编码的分词器
    args=training_arguments,  # 训练参数
    packing=packing,  # 打包配置（用于更高效的训练）
)


In [ ]:
# LoRA注意力维度
lora_r = 64

# LoRA缩放的Alpha参数
lora_alpha = 16

# LoRA层的丢弃概率
lora_dropout = 0.1

# 激活4位精度的基础模型加载
use_4bit = True

# 计算4位基础模型的数据类型
bnb_4bit_compute_dtype = "float16"

# 量化类型（fp4或nf4）
bnb_4bit_quant_type = "nf4"

# 为4位基础模型激活嵌套量化（双重量化）
use_nested_quant = False

# 加载LoRA配置
peft_config = LoraConfig(
    lora_alpha=lora_alpha,  # LoRA的缩放系数Alpha
    lora_dropout=lora_dropout,  # LoRA层的丢弃概率
    r=lora_r,  # LoRA的注意力维度
    bias="none",  # 偏置配置
    task_type="CAUSAL_LM",  # 任务类型为因果语言模型
)

# 使用QLoRA配置加载模型
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,  # 是否以4位精度加载模型
    bnb_4bit_quant_type=bnb_4bit_quant_type,  # 4位量化的类型
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,  # 4位模型的计算数据类型
    bnb_4bit_use_double_quant=use_nested_quant,  # 是否使用嵌套量化
)

# 从预训练加载模型，并配置设备映射
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # 量化配置
    device_map=device_map  # 设备映射配置
)

# 创建SFT（Structured Fine-Tuning）训练器
trainer = SFTTrainer(
    model=model,  # 指定模型
    train_dataset=dataset,  # 训练数据集
    peft_config=peft_config,  # PEFT（Parameter-Efficient Fine-Tuning）配置，此处为LoRA配置
    dataset_text_field="text",  # 数据集中文本字段的名称
    max_seq_length=max_seq_length,  # 最大序列长度
    tokenizer=tokenizer,  # 用于文本编码的分词器
    args=training_arguments,  # 训练参数
    packing=packing,  # 打包配置（用于更高效的训练）
)


### 三种方法的对比

1. **实现区别**:
   - **Prefix-Tuning**：通过在模型的输入端添加一组可训练的向量（前缀）来调整模型的输出，不直接改变模型的权重。
   - **LoRA**：通过向模型的线性层添加低秩结构（而不是替换原有权重）来微调模型，保持大部分预训练权重不变。
   - **QLoRA**：对预训练模型进行4位高精度量化，添加少量可学习的低秩适配器（LoRA）权重，通过反向传播梯度调整这些权重。

     
<div class="alert alert-warning">
<b>Adapter、Prefix-Tuning、LoRA、QLoRA等参数有效性学习方法（PEFT）本质上都是插入少量的新的参数，这些新的参数可以对预训练模型起到提示作用，只不过并不是以人类可读的离散的模板形式体现而已 。<b>
</div>

2. **优缺点**:
   - **Prefix-Tuning**：
     - 优点：灵活性高，适用于各种大小的模型，不改变原始模型。
     - 缺点：可能在复杂任务上不如直接微调模型有效。
   - **LoRA**：
     - 优点：有效地微调大型模型而不需要大量资源，保持大部分预训练权重不变。
     - 缺点：可能无法在所有任务上达到完全微调的效果。
   - **QLoRA**：
     - 优点：显著降低微调大型模型的内存需求，保持良好性能。
     - 缺点：可能在特定场景下不如完全微调模型有效。

3. **使用情况**:
   - **QLoRA**：在资源受限但需要微调大型模型以保持或接近其原始性能的情况下使用。
   - **LoRA**：在希望微调大型模型但限于资源，同时希望保持大部分预训练权重不变的情况下使用。
   - **Prefix-Tuning**：在需要快速适应新任务，同时希望保持原始模型不变的情况下使用，适用于各种规模的模型。



# 六、可能有害的全量微调

最后，我们介绍一篇相当有意思的文章，该文章的内容对我们使用指令微调将预训练模型改造为Chat模型和下游专业模型有一定指导意义。该文章的研究表明，对于经过**充分预训练**的模型，使用通用指令微调数据集进行**全量微调**可能对模型的效果有害。LoRA指令微调虽然在学习知识方面有限，但能有效保持模型的格式输出能力，且不会引入过多无关词汇，避免“幻觉”现象的发生。

## 充分预训练后：Lora微调有局限，但很有用
许许多多的大模型初学者都会遇到一个问题：如何在不损失模型性能的情况下，将模型应用到自己的任务中。这个问题的答案通常是微调。更为具体地说，通常是Lora微调，相比于全量微调，Lora微调可以更快地适应新任务，同时减少了计算资源的消耗。然而，研究表明，Lora微调不能使得模型学习到新的知识，而只是使得模型能够按照期望的格式输出。换句话说，经过Lora微调的模型仅仅是提升了更加充分地利用了自己预训练的知识，而并没有学到新的知识。

### LoRA仅能够让模型学会输出的格式
文章通过分析微调后模型的输出概率分布与预训练模型的输出概率分布的差异来衡量模型是否通过指令微调学习到了知识。具体来说，作者通过计算KL散度来衡量两个概率分布的差异。

实验结果表明，模型的概率分布偏移的并不大。模型仅在前百分之五的概率分布中有比较大的KL散度发散，而在余下的概率分布中几乎保持不变，并且与全量训练相比，LoRA训练的KL散度偏移接近于0。这说明LoRA仅仅做到了学会输出的格式，而做不到学会具体的知识。体现在loss上我们可以发现，使用LoRA训练时模型收敛的非常快，然而在快速收敛之后loss保持平稳，无法进行进一步的下降。

<img src="image/youhai_1.png"  width="600" height="400" alt="rag-vs-finetuning-chn"/>

### 增大数据集规模并不能解决问题
为了验证LoRA训练的模型是否真的没有学到新的知识，现在许多研究将下游训练的指令微调数据集扩大到百万级的规模，这种做法并不能进一步提高模型的性能。即使将数据集的规模扩大52倍；扩大326倍，也没有作用。如下图所示，扩大数据集规模后LoRA训练的模型在五个维度上的表现都没有得到增强。

<img src="image/youhai_2.png"  width="800" height="400" alt="rag-vs-finetuning-chn"/>

### 虽然LoRA微调有局限，但比全量微调强
研究人员发现，全量微调会学习指令微调数据集中的用词导致严重的幻觉。团队研究了全量微调和LoRA微调后模型输出概率分布中的边缘偏移token和偏移token。发现LoRA训练后的偏移token常常为风格token，例如However和Typically。而全量微调中的偏移token包含了指令微调数据集中出现的所有token，也就是说全量微调可能会把指令微调数据集中的任何token利用到测试场景中，即使这些token与测试场景无关。

<img src="image/youhai_3.png"  width="600" height="400" alt="rag-vs-finetuning-chn"/>

例如，上图的测试场景的提问为是什么导致了极光，而全量微调的模型大量使用了指令微调数据集中问题为“哪里能看到极光“的样本中的token，这导致了输出的内容偏离了实际的提问，而LoRA训练的模型则正确的回答了该问题。

### 结论
LoRA微调虽然不能使得模型学会新的知识，但是它可以使得模型更好地利用自己预训练的知识，同时避免了全量微调中可能出现的幻觉现象。在特定的指令数据集上，全量微调可能会学到一些无关的知识，并且这些无关的知识可能会对模型的性能产生严重的负面影响。因此，LoRA微调虽然有局限，但是通常是更好的选择。



## 总结

一、大模型微调简介
- 微调解决的问题
- 微调到底是什么

二、微调示例
- 微调步骤
- 微调过程展示

三、微调模型数据准备
- 数据收集
- 数据预处理

四、微调在模型训练中的位置
- 大模型训练流程
- 微调技术与提示工程对比

五、PEFT方法对比
- Prefix Tuning
- LoRA
- QLoRA

六、可能有害的全量微调
- Lora微调有局限，但很有用
- 全量微调更加糟糕

reference:
- https://blog.csdn.net/weixin_39653948/article/details/132954262 概览
- https://www.bilibili.com/video/BV1ry4y1F7QS?p=2&vd_source=ffb57dc7e05daab8f33d2ea7860289d0 概览
- https://zhuanlan.zhihu.com/p/644122818 微调方法
- https://medium.com/@masteringllm/llm-training-a-simple-3-step-guide-you-wont-find-anywhere-else-98ee218809e5 训练过程
- https://cloud.tencent.com/developer/article/2315386 训练过程
- https://arxiv.org/pdf/2106.09685.pdf Lora
- https://arxiv.org/pdf/1902.00751.pdf Parameter-Efficient Transfer Learning for NLP 
- https://arxiv.org/pdf/2101.00190.pdf Prefix Tuning
- https://arxiv.org/pdf/2103.10385.pdf P-Tuning
- https://arxiv.org/pdf/2104.08691.pdf Prompt Tuning
- https://arxiv.org/pdf/2303.15647.pdf PEFT survey
- https://arxiv.org/pdf/2312.12148.pdf PEFT survey 2